In [ ]:
from importlib import reload
import numpy as np
import os
import random
import pickle
import multiprocessing
import functools
import seaborn
import logging
import typhon
import cmocean
from pylab import cm
from os.path import join
from netCDF4 import Dataset
from tqdm import tqdm_notebook
from scipy.interpolate import interp1d
from scipy.signal import detrend
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import processing_tools as ptools
import analysis_tools as atools
from importlib import reload
from moisture_space import utils, plots
from moisture_space import moisture_space
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

# Config

In [ ]:
models = ['ICON', 'NICAM', 'GEOS', 'IFS', 'MPAS', 'FV3', 'UM', 'SAM', 'ARPEGE', 'ERA5']
runs = {
    'ICON': ['2.5km', '5.0km_1'],
    'NICAM': ['3.5km'],
    'SAM': ['4.0km'],
    'UM': ['5.0km'],
    'FV3': ['3.25km'],
    'GEOS': ['3.0km'],
    'IFS': ['4.0km', '9.0km'],
    'MPAS': ['3.75km'],
    'ARPEGE': ['2.5km'],
    'ERA5': ['31.0km']
}
exps = []
for m in models:
    for run in runs[m]:
        exps.append(m+'-'+run)
        
time_period = ['0810', '0908']
variables_3D = ['TEMP', 'PRES', 'QV', 'QI', 'QC', 'RH', 'ICQI', 'CFI', 'ICQC', 'CFL', 'W']
variables_2D = ['IWV', 'H_tropo']
datapath = '/mnt/lustre02/work/mh1126/m300773/DYAMOND/{}/random_samples/'
filenames = '{}-{}_{}_sample_{}_{}-{}.nc'
num_profiles = int(1 * 1e7)
perc_values = np.arange(1, 100.5, 1.0)
num_percs = len(perc_values)
iwv_bin_bnds = np.arange(0, 101, 1)
bins = range(len(iwv_bin_bnds) - 1) 
exp = 0
h = np.arange(100, 17900, 400) # height vector to interpolate all models on
height = {}
num_levels = {}
for m in models:
    for run in runs[m]:
        key = m+'-'+run
        filename = filenames.format(m, run, variables_3D[0], num_profiles, time_period[0], time_period[1])
        if run == '2.5km_winter':
            filename = filenames.format(m, run, variables_3D[0], num_profiles, '0120', '0202')
        filename = join(datapath.format(m), filename)
        #xarr = xr.open_dataset(filename)
        with(Dataset(filename)) as ds:
            height[key] = ds.variables['height'][:].filled(np.nan)
        num_levels[key] = len(height[key])
        
# NICAM excluded for variablilty analysis!
variability_exclude = ['']
models_variability = [m for m in exps if not m in variability_exclude]

# Read randomly sampled profiles from files and interpolate to common height

In [ ]:
reload(moisture_space)
ms_perc = {}
ms_bins = {}
percentiles = {}
for m in models:
    for run in runs[m]:
        exp = m+'-'+run
        path_perc = f'{m}-{run}_{time_period[0]}-{time_period[1]}_perc_means_{num_profiles}_1exp.pkl'
        path_bin = f'{m}-{run}_{time_period[0]}-{time_period[1]}_bin_means_{num_profiles}_1exp.pkl'
        if run == '2.5km_winter':
            path_perc = f'{m}-{run}_0120-0202_perc_means_{num_profiles}_1exp.pkl'
            path_bin = f'{m}-{run}_0120-0202_bin_means_{num_profiles}_1exp.pkl'
        with open(join(datapath.format(m), path_perc), 'rb' ) as infile:
            perc = pickle.load(infile)
        with open(join(datapath.format(m), path_bin), 'rb' ) as infile:
            binned = pickle.load(infile)
        percentiles[exp] = perc['percentiles']
        if 'OLR' in variables_2D and 'STOA' in variables_2D:
            if m == 'ICON':
                perc['mean']['OLR'] *= -1 
                binned['mean']['OLR'] *= -1
            if np.mean(perc['mean']['STOA']) < 0:
                perc['mean']['STOA'] *= -1
                binned['mean']['STOA'] *= -1
        ms_perc[exp] = {}
        ms_bins[exp] = {}
        for var in variables_3D:
            stats_perc = moisture_space.ProfileStats.from_dict(perc, var)
            ms_perc[exp][var] = moisture_space.PercMoistureSpace(exp, stats_perc, perc_values, height[exp]).interpolate(h)
            stats_bin = moisture_space.ProfileStats.from_dict(binned, var)
            ms_bins[exp][var] = moisture_space.BinMoistureSpace(exp, stats_bin, perc_values, height[exp], binned['count']).interpolate(h)
            ms_bins[exp][var].remove_empty_bins(number_threshold=300)
        for var in variables_2D:
            stats_perc = moisture_space.ProfileStats.from_dict(perc, var)
            ms_perc[exp][var] = moisture_space.PercMoistureSpace(exp, stats_perc, perc_values, None)   
            stats_bin = moisture_space.ProfileStats.from_dict(binned, var)
            ms_bins[exp][var] = moisture_space.BinMoistureSpace(exp, stats_bin, perc_values, None, binned['count'])
            ms_bins[exp][var].remove_empty_bins(number_threshold=300)

# Calculate lapse rate and total cloud condensate

In [ ]:
reload(utils)
Rd = typhon.constants.gas_constant_dry_air
cp = typhon.constants.isobaric_mass_heat_capacity
g = typhon.constants.g
for exp in exps:
    lapse_rate = -np.gradient(ms_perc[exp]['TEMP'].mean, axis=1) / np.gradient(h)
    stability = Rd / cp * ms_perc[exp]['TEMP'].mean / ms_perc[exp]['PRES'].mean * (1 - lapse_rate * cp / g)
    cc_tot = ms_perc[exp]['QI'].mean + ms_perc[exp]['QC'].mean
    e = typhon.physics.specific_humidity2vmr(ms_perc[exp]['QV'].mean) * ms_perc[exp]['PRES'].mean
    es = typhon.physics.e_eq_mixed_mk(ms_perc[exp]['TEMP'].mean)
    td = utils.calc_dewpoint(e)
    density = utils.calc_density_moist_air(
        ms_perc[exp]['PRES'].mean,
        ms_perc[exp]['TEMP'].mean,
        ms_perc[exp]['QV'].mean
    )
    w_int = utils.calc_integrated_velocity(
        ms_perc[exp]['W'].mean,
        h,
        100,
        12500,
        ms_perc[exp]['TEMP'].mean,
        ms_perc[exp]['PRES'].mean,
        ms_perc[exp]['QV'].mean
    )
    theta_e = utils.calc_equivalent_pot_temp(
        ms_perc[exp]['TEMP'].mean,
        ms_perc[exp]['PRES'].mean,
        ms_perc[exp]['QV'].mean
    )
    theta_es = utils.calc_sat_equivalent_pot_temp(
        ms_perc[exp]['TEMP'].mean,
        ms_perc[exp]['PRES'].mean
    )
    stats_perc = moisture_space.ProfileStats(variable='LR', mean=lapse_rate)
    ms_perc[exp]['LR'] = moisture_space.PercMoistureSpace(exp, stats_perc, ms_perc[exp]['TEMP'].bins, h)
    stats_perc = moisture_space.ProfileStats(variable='THETA_E', mean=theta_e)
    ms_perc[exp]['THETA_E'] = moisture_space.PercMoistureSpace(exp, stats_perc, ms_perc[exp]['TEMP'].bins, h)
    stats_perc = moisture_space.ProfileStats(variable='THETA_ES', mean=theta_es)
    ms_perc[exp]['THETA_ES'] = moisture_space.PercMoistureSpace(exp, stats_perc, ms_perc[exp]['TEMP'].bins, h)
    stats_perc = moisture_space.ProfileStats(variable='S', mean=stability)
    ms_perc[exp]['S'] = moisture_space.PercMoistureSpace(exp, stats_perc, ms_perc[exp]['TEMP'].bins, h)
    stats_perc = moisture_space.ProfileStats(variable='QTOT', mean=cc_tot)
    ms_perc[exp]['QTOT'] = moisture_space.PercMoistureSpace(exp, stats_perc, ms_perc[exp]['TEMP'].bins, h)
    stats_perc = moisture_space.ProfileStats(variable='E', mean=e)
    ms_perc[exp]['E'] = moisture_space.PercMoistureSpace(exp, stats_perc, ms_perc[exp]['TEMP'].bins, h)
    stats_perc = moisture_space.ProfileStats(variable='TEMP_D', mean=td)
    ms_perc[exp]['TEMP_D'] = moisture_space.PercMoistureSpace(exp, stats_perc, ms_perc[exp]['TEMP'].bins, h)
    stats_perc = moisture_space.ProfileStats(variable='Es', mean=es)
    ms_perc[exp]['Es'] = moisture_space.PercMoistureSpace(exp, stats_perc, ms_perc[exp]['TEMP'].bins, h)
    stats_perc = moisture_space.ProfileStats(variable='logQV', mean=np.log(ms_perc[exp]['QV'].mean))
    ms_perc[exp]['logQV'] = moisture_space.PercMoistureSpace(exp, stats_perc, ms_perc[exp]['TEMP'].bins, h)
    stats_perc = moisture_space.ProfileStats(variable='Wint', mean=w_int)
    ms_perc[exp]['Wint'] = moisture_space.PercMoistureSpace(exp, stats_perc, ms_perc[exp]['TEMP'].bins, h)
    stats_perc = moisture_space.ProfileStats(variable='DENS', mean=density)
    ms_perc[exp]['DENS'] = moisture_space.PercMoistureSpace(exp, stats_perc, ms_perc[exp]['TEMP'].bins, h)

# Create collections of moisture spaces for each variable

In [ ]:
reload(moisture_space)
ms_perc_series = {}
ms_bins_series = {}
for var in variables_3D+['QTOT', 'E', 'Es', 'logQV', 'LR', 'S', 'TEMP_D']:
    perc_space_list = [ms_perc[exp][var] for exp in models_variability]
    ms_perc_series[var] = moisture_space.MoistureSpaceSeries(perc_space_list)
for var in variables_3D:
    bin_space_list = [ms_bins[exp][var] for exp in models_variability]
    ms_bins_series[var] = moisture_space.MoistureSpaceSeries(bin_space_list, remove_nans=True)

# Calculate EOFs and expansion coefficients

eofs = {}
eofs_iwv = {}
variability_frac = {}
variability_frac_iwv = {}
expansion_coeffs = {}
expansion_coeffs_iwv = {}
rep_1 = {}
rep_2 = {}
rep_5 = {}
for var in ['RH', 'TEMP']:
    eofs[var], variability_frac[var], expansion_coeffs[var] = ms_perc_series[var].calc_EOFs()
    eofs_iwv[var], variability_frac_iwv[var], expansion_coeffs_iwv[var] = ms_bins_series[var].calc_EOFs()
    rep_1[var] = ms_perc_series[var].recunstruct_from_EOFs(1)
    rep_2[var] = ms_perc_series[var].recunstruct_from_EOFs(2)
    rep_5[var] = ms_perc_series[var].recunstruct_from_EOFs(5)

# Calculate EOFs for several variables

In [ ]:
reload(moisture_space)
var_pairs = [('RH', 'QI', 'QC', 'W')]
ms_perc_collections = {}
eofs_cov = {}
variability_frac_cov = {}
expansion_coeffs_cov = {}
for pair in var_pairs:
    ms_perc_collections[pair] = moisture_space.MoistureSpaceSeriesCollection([ms_perc_series[pair[i]] for i in range(len(pair))])
    eofs_cov[pair], variability_frac_cov[pair], expansion_coeffs_cov[pair] = ms_perc_collections[pair].calc_EOFs()
    #eofs_combined = ms_perc_collections[pair].calc_EOFs()

# Perform SVD for Pairs of Variables

In [ ]:
reload(moisture_space)
var_pairs = [('RH', 'TEMP'), ('RH', 'S'), ('RH', 'logQV'), ('TEMP', 'logQV'), ('RH', 'QTOT'), ('RH', 'QC'), ('RH', 'QV'), ('RH', 'QI'), ('RH', 'CFI'), ('QV', 'QTOT'), ('QI', 'QC'), ('TEMP', 'W'), ('TEMP', 'QV')]
ms_perc_pairs = {}
singular_vec = {}
frac_variability = {}
expansion_coeffs_svs = {}
for pair in var_pairs:
    ms_perc_pairs[pair] = moisture_space.MoistureSpaceSeriesPair(ms_perc_series[pair[0]], ms_perc_series[pair[1]])
    singular_vec_0, singular_vec_1, frac_variability[pair],\
    expansion_coeffs_0, expansion_coeffs_1 = ms_perc_pairs[pair].perform_SVD()
    singular_vec[pair] = (singular_vec_0, singular_vec_1)
    expansion_coeffs_svs[pair] = (expansion_coeffs_0, expansion_coeffs_1)

# Plots

## Fraction of variability explained by EOFs

In [ ]:
plt.rcParams.update({'font.size': 15})
plot_vars = ['RH', 'TEMP']
fig, ax = plt.subplots(1, len(plot_vars), figsize=(15, 6))
for i, var in enumerate(plot_vars):
    ax[i].plot(variability_frac[var], '.')
    ax[i].set_xlabel(f'{var} EOF Nr.')
    ax[i].set_ylabel('Fraction of variability explained by EOF')
plt.tight_layout()

## EOFs

In [ ]:
plot_vars = ['RH', 'TEMP']
num_eofs = 3
contours = np.arange(-0.06, 0.0625, 0.0025)
x_lims = [0, 100]
y_lims = [0, 17.5]
x_label = 'Percentile of IWV'
y_label = 'Height [km]'

fig, ax = plt.subplots(len(plot_vars), num_eofs, figsize=(6 * num_eofs, 5 * len(plot_vars)), sharey=True)

for i, var in enumerate(plot_vars):
    for j in range(num_eofs):
        plots.moisture_space_contourf(fig, ax[i, j], perc_values, h * 1e-3, eofs[var][j].T, contours,\
                              x_lims, y_lims, x_label, '', 'Variance of relative humidity',\
                              cmap='difference')
        ax[i, j].set_title(f'{var} EOF {j}')
plt.tight_layout()
for i in range(len(plot_vars)):
    ax[i, 0].set_ylabel(y_label)

## Expansion coefficients

In [ ]:
plot_vars = ['RH', 'TEMP']
num_eofs = 3
contours = np.arange(-0.06, 0.065, 0.005)
x_lims = [0, 100]
y_lims = [0, 20]
x_label = 'Percentile of IWV'
y_label = 'Height [km]'

fig, ax = plt.subplots(len(plot_vars), num_eofs, figsize=(5 * num_eofs, 4 * len(plot_vars)))
ax = ax.ravel()
k = 0
for i, var in enumerate(plot_vars):
    for j in range(num_eofs):
        ax[k].plot(expansion_coeffs[var][j], '.', markersize=20)
        ax[k].plot([-1, 7], [0, 0], '--', color='k')
        ax[k].set_ylim(-4, 4)
        ax[k].set_xticks(np.arange(len(models_variability)))
        ax[k].set_xticklabels(models_variability, fontsize=12, rotation=90)
        ax[k].set_title(f'Principal components {var} EOF {j}', fontsize=13)
        ax[k].set_ylabel('Principal component')
        k += 1
        seaborn.despine()
plt.tight_layout()

# EOFs for IWV-binned moisture space

## Fraction of variability explained by EOF

In [ ]:
plot_var = 'RH'
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(variability_frac_iwv[plot_var], '.')
ax.set_xlabel(f'{plot_var} EOF Nr.')
ax.set_ylabel('Fraction of variability explained by EOF')
plt.tight_layout()

## EOFs

In [ ]:
plot_vars = ['RH']
num_eofs = 3
contours = np.arange(-0.06, 0.0625, 0.0025)
x_lims = [5, 75]
y_lims = [0, 17.5]
x_label = 'IWV'
y_label = 'Height [km]'

fig, ax = plt.subplots(len(plot_vars), num_eofs, figsize=(6 * num_eofs, 5 * len(plot_vars)))
ax = ax.ravel()
k = 0
for i, var in enumerate(plot_vars):
    for j in range(num_eofs):
        plots.moisture_space_contourf(fig, ax[k], ms_bins_series[var].bins, h * 1e-3, eofs_iwv[var][j].T, contours,\
                              x_lims, y_lims, x_label, y_label, 'Standard deviation of relative humidity [%]',\
                              cmap='difference')
        ax[k].set_title(f'{var} EOF {j}')
        k += 1
plt.tight_layout()

## Expansion coefficients

In [ ]:
plot_vars = ['RH']
num_eofs = 3
contours = np.arange(-0.06, 0.065, 0.005)
x_lims = [0, 100]
y_lims = [0, 20]
x_label = 'Percentile of IWV'
y_label = 'Height [km]'

fig, ax = plt.subplots(len(plot_vars), num_eofs, figsize=(5 * num_eofs, 4 * len(plot_vars)))
ax = ax.ravel()
k = 0
for i, var in enumerate(plot_vars):
    for j in range(num_eofs):
        ax[k].plot(expansion_coeffs_iwv[var][j], '.', markersize=20)
        ax[k].plot([-1, 7], [0, 0], '--', color='k')
        ax[k].set_ylim(-4, 4)
        ax[k].set_xticks(np.arange(len(models_variability)))
        ax[k].set_xticklabels(models_variability, fontsize=12, rotation=90)
        ax[k].set_title(f'Principal components {var} EOF {j}')
        ax[k].set_ylabel('Principal component')
        k += 1
        seaborn.despine()
plt.tight_layout()

# EOFs for several variables

## EOFs

In [ ]:
plt.rcParams.update({'font.size': 13})
reload(plots)
pair = ('RH', 'QI', 'QC', 'W')
num_vars = len(pair)
num_eofs = 4
contours = {
    'RH': np.arange(-0.06, 0.0625, 0.0025),
    'QI': np.arange(-0.00001, 0.0000101, 0.000001),
    'QC': np.arange(-0.00001, 0.0000101, 0.000001),
    'W': np.arange(-0.005, 0.005, 0.0005)
}
x_lims = [0, 100]
y_lims = [0, 17.5]

x_label = 'Percentile of IWV'
y_label = 'Height [km]'
fig, ax = plt.subplots(len(pair), num_eofs, figsize=(15, 12), sharey=True, sharex=True)
for j in range(len(pair)):
    for i in range(num_eofs):
        plots.moisture_space_contourf(fig, ax[i, j], perc_values, h * 1e-3, eofs_cov[pair][j][i].T, contours[pair[j]],\
                              x_lims, y_lims, '', '', '', cm_orientation='vertical',\
                              cmap='difference')
        ax[i, j].set_title(f'{pair[j]} EOF {i}')
plt.tight_layout()
for i in range(num_eofs):
    ax[num_vars-1, i].set_xlabel(x_label)
for i in range(num_vars):
    ax[i, 0].set_ylabel(y_label)

## Principal components

In [ ]:
len(expansion_coeffs_cov[pair])

fig, ax = plt.subplots(len(pair), num_eofs, figsize=(15, 15))
for j in range(len(pair)):
    for i in range(num_eofs):
        ax[i, j].plot(expansion_coeffs_cov[pair][j][i], '.', markersize=12)
        ax[i, j].plot([0, len(models_variability)], [0, 0], '--', color='k')
        ax[i, j].set_xticks(np.arange(len(models_variability)))
        ax[i, j].set_xticklabels(models_variability, rotation=90)
        ax[i, j].set_title(f'PC {pair[j]} EOF {i}')
plt.tight_layout()

expansion_coeffs_cov[pair]

## Fraction of variability explained by EOFs

plt.plot(variability_frac_cov[pair], '.')

# SVDs

## Singular vectos

In [ ]:
reload(plots)
plt.rcParams.update({'font.size': 15})
pair = ('RH', 'QI')
num_svs = 3
contours = np.arange(-0.06, 0.0625, 0.0025)
contours_qi = np.arange(-0.2, 0.21, 0.01)
x_lims = [0, 100]
y_lims = [0, 20]
x_label = 'Percentile of IWV'
y_label = 'Height [km]'

fig, ax = plt.subplots(2, num_svs, figsize=(6 * num_svs, 10))

for i in range(2):
    for j in range(num_svs):
        if i == 1 and pair[i] in ['QI', 'QC']:
            plots.moisture_space_contourf(fig, ax[i, j], perc_values, h * 1e-3, singular_vec[pair][i][j].T, contours_qi,\
                                  x_lims, y_lims, x_label, y_label, 'Singular vector',\
                                  cmap='difference')
        else:
            plots.moisture_space_contourf(fig, ax[i, j], perc_values, h * 1e-3, singular_vec[pair][i][j].T, contours,\
                                  x_lims, y_lims, x_label, y_label, 'Singular vector',\
                                  cmap='difference')
        ax[i, j].set_title(f'SV {j} {pair[i]}')

plt.tight_layout()

## Fraction of co-variability explained by singular vectors

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(frac_variability[pair], '.')
ax.set_xlabel(f'{pair[0]}-{pair[1]} SV Nr.')
ax.set_ylabel('Fraction of variability explained by SV')

## Expansion coefficients

In [ ]:
num_svs = 3
fig, ax = plt.subplots(2, num_svs, figsize=(6 * num_svs, 12))
for i in range(2):
    for j in range(num_svs):
        ax[i, j].plot(expansion_coeffs_svs[pair][i][:, j], '.', markersize=15)
        ax[i, j].plot([-1, 7], [0, 0], '--', color='k')
        #ax[k].set_ylim(-4, 4)
        ax[i, j].set_xticks(np.arange(len(models_variability)))
        ax[i, j].set_xticklabels(models_variability, rotation=90)
        ax[i, j].set_title(f'Expansion coefficients {pair[i]} SV {j}')
plt.tight_layout()